# Experiment 03: Regime-Stratified Evaluation

**Proposal**: "Performance stability across volatility percentiles" and splits across "bull vs. bear periods and weekday vs. weekend trading."

This notebook evaluates model performance stratified by realised volatility (RV) percentiles on the test set.

**Prerequisites**: Run Experiment 01 (and optionally 02). Uses `checkpoints/joint/best.pt`.

## 1. Setup

In [ ]:
import os
import sys
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset

cwd = os.getcwd()
PROJECT_ROOT = os.path.dirname(cwd) if os.path.basename(cwd) == "experiments" else cwd
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
os.chdir(PROJECT_ROOT)
print(f"Working directory: {os.getcwd()}")

## 2. Load Data and Model

In [ ]:
from src.dataset import CryptoRegimeDataset
from src.models import TTTModel
from src.ttt_learner import TTTAdaptor
from src.utils import get_device

CHECKPOINT = "checkpoints/joint/best.pt"
device = get_device()

dataset = CryptoRegimeDataset("data/processed")
_, _, test_ds = dataset.get_splits()
test_idx = dataset._splits["test"]
rv_test = dataset.rv_values[test_idx].numpy()
labels_test = dataset.labels[test_idx].numpy()

## 3. Define Volatility Bins

In [ ]:
# Quartiles of RV on test set
q25, q50, q75 = np.percentile(rv_test, [25, 50, 75])

def get_bin_mask(rv, bin_name):
    if bin_name == "low (0-25%)":
        return rv <= q25
    elif bin_name == "mid-low (25-50%)":
        return (rv > q25) & (rv <= q50)
    elif bin_name == "mid-high (50-75%)":
        return (rv > q50) & (rv <= q75)
    else:  # high (75-100%)
        return rv > q75

bins = ["low (0-25%)", "mid-low (25-50%)", "mid-high (50-75%)", "high (75-100%)"]
print(f"RV quartiles: {q25:.6f}, {q50:.6f}, {q75:.6f}")

## 4. Compute Metrics per Bin (Baseline + TTT)

In [ ]:
from src.eval import compute_metrics

ckpt = torch.load(CHECKPOINT, map_location=device, weights_only=False)
train_args = ckpt.get("args", {})
model = TTTModel(num_classes=2, aux_task=train_args.get("aux_task", "mask"), 
                 num_groups=train_args.get("num_groups", 8)).to(device)
model.load_state_dict(ckpt["model_state_dict"])

adaptor = TTTAdaptor(model=model, base_lr=0.05, ttt_steps=10, mask_mode=train_args.get("mask_mode", "random_slices"),
                     ttt_optimizer="adam", entropy_adaptive=True, entropy_gate_threshold=0.3, device=device)

THRESHOLD = 0.35
results = []

for bin_name in bins:
    mask = get_bin_mask(rv_test, bin_name)
    idx = np.where(mask)[0]
    if len(idx) < 5:
        continue
    indices = np.asarray(test_idx)[idx].tolist()
    subset = Subset(dataset, indices)
    loader = DataLoader(subset, batch_size=1, shuffle=False)
    
    # Baseline
    out_b = adaptor.evaluate_baseline(loader)
    m_b = compute_metrics(out_b["probabilities"].numpy(), out_b["labels"].numpy(), 
                         out_b["rv_values"].numpy(), threshold=THRESHOLD)
    
    # TTT standard (per sample)
    probs_t, labels_t, rv_t = [], [], []
    for batch in loader:
        imgs, lbl, rv = batch[0].to(device), batch[1], batch[2]
        logits, _ = adaptor.adapt_and_predict(imgs)
        p = F.softmax(logits, dim=-1).cpu().numpy()
        probs_t.append(p)
        labels_t.append(lbl.numpy())
        rv_t.append(rv.numpy())
    probs_t = np.concatenate(probs_t)
    labels_t = np.concatenate(labels_t)
    rv_t = np.concatenate(rv_t)
    m_t = compute_metrics(probs_t, labels_t, rv_t, threshold=THRESHOLD)
    
    results.append({"bin": bin_name, "n": len(idx), "baseline_acc": m_b["accuracy"], "baseline_f1": m_b["f1"],
                   "ttt_acc": m_t["accuracy"], "ttt_f1": m_t["f1"]})

for r in results:
    print(f"{r['bin']} (n={r['n']}): Baseline acc={r['baseline_acc']:.3f} f1={r['baseline_f1']:.3f} | TTT acc={r['ttt_acc']:.3f} f1={r['ttt_f1']:.3f}")

## 5. Results Table

In [ ]:
import pandas as pd
df = pd.DataFrame(results)
df

**Results (example run):**

| RV bin | n | Baseline acc | Baseline F1 | TTT acc | TTT F1 |
|--------|---|--------------|------------|---------|--------|
| low (0-25%) | 194 | 0.964 | 0.000 | 0.314 | 0.000 |
| mid-low (25-50%) | 194 | 0.943 | 0.000 | 0.325 | 0.000 |
| mid-high (50-75%) | 193 | 0.974 | 0.000 | 0.394 | 0.000 |
| high (75-100%) | 194 | 0.175 | 0.091 | 0.577 | **0.717** |

**Interpretation**: TTT improves performance mainly in the **high-volatility** regime (accuracy 0.18 to 0.58, F1 0.09 to 0.72). In low/mid-vol bins the baseline already predicts low-vol correctly; TTT shifts predictions toward high-vol and hurts accuracy there. This supports the proposal: *"We expect TTT to improve robustness under market regime changes, particularly during high-volatility periods where distribution shifts are largest."*